# LLM examples

Learning goals: 

- Practice different introductory use cases of LLMs

## Setup

- To run this notebook you will need to create an account in cohere.ai, and get your API Key.
- You also need to install the python package `cohere`.

## Why cohere? 

- It was one of the first providers of LLMs as a service, was already in the market before ChatGPT. 

- Easy to use Python SDK. 

- A free account gives you access to a Trial API key, which allows 100 calls per minute. 

- Similar in structure to other APIs and frameworks (if you learn this, you'll navigate the others easily). 

## Preparing the work

Let's initialize the environment:

In [3]:
import cohere

/Users/joseantonio.rodriguez15/code/PDAI25/env_proto/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Log in to cohere, navigate to DASHBOARD (at the top), then to API Keys (at the sidebar)

# Never store your key in the code or push it to your repository!!!

with open("cohere.key") as f:
    COHERE_API_KEY = f.read()
cohere_client = cohere.ClientV2(COHERE_API_KEY)

## Use case 1: Prompt templates

### Prompt templates from scratch

We can use LLMs to take actions for us and avoid the alternative of writing code. 

The example below will ask for a text input by the user. Once introduced, it will classify the language of the text. 

In [20]:
prompt_template = "Return the language of the text between triple quotation marks: ```!text!``` " 

user_text = input("Enter some text")
final_prompt = prompt_template.replace("!text!", user_text)

formatted_message = {
    'role': 'user',
    'content': final_prompt 
}

response = cohere_client.chat(
    model='command-r-plus',
    messages=[formatted_message],
)

language = response.message.content[0].text
print(language)

The language of the text is English.


In [24]:
response.dict()

{'id': '270c5101-5551-497a-9089-75f064129574',
 'finish_reason': 'COMPLETE',
 'message': {'role': 'assistant',
  'content': [{'type': 'text',
    'text': 'The language of the text is English.'}]},
 'usage': {'billed_units': {'input_tokens': 21.0, 'output_tokens': 8.0},
  'tokens': {'input_tokens': 215.0, 'output_tokens': 8.0}}}

You might need a more specific prompt, like one that includes "reply only with a single word containing the name of the recognized language" (try that as an exercise). 

The typical workflow would be to use that output as part of the business logic.

In [25]:
if language == "Spanish":
    # Take actions... 
    pass

### Using a system prompt

All LLMs have converged to a common design where they use roles. Roles can be "system" (describing how the assistant should behave), and "user" (describing user questions). A reserved role is "assistant" (indicating a response by the assistant).

This facilitates the usage of LLMs as instruction models and provides a common mechanism for prompting.

In [31]:
system_prompt = {
    'role': 'system',
    'content': """
        You are a system that classifies the language of the text inputted by the user.
        You should read the user input, infer the language in which it is written, and output it.
        Respond with a single word indicating the language in upper case.
        Respond: UNKNOWN if it is not clear.
        """
}

user_text = input("Enter some text")

user_text = {
    'role': 'user',
    'content': user_text
}

response = cohere_client.chat(
    messages = [system_prompt, user_text],
    model='command-r-plus'
)

response.message.content[0].text

'FRENCH'

### In-prompt examples

We can use LLMs to "simulate" the way AI systems work and provide "examples" of what we want to achieve, as part of the prompt. 

This is know as "few-shot", "in-prompt" learning. 

The example below tries to classify the restaurant type based on its name. It does so by preparing a "template" of known inputs and outputs, then asking to complete a last input: 

In [38]:
prompt2="""
Classify the restaurant type based on the restaurant name. 
Complete the sequence as in the following examples:
Name: "Leche de Tigre" Type: "Peruvian"
Name: "Le Parisien" Type: "French"
Name: "Tapas & Fiesta" Type: "Spanish"
Name: "Rodizio Grill" Type: "Brazilian"
Name: "Shenzen Garden" Type: "Chinese"
Name: "Bella Napoli" Type: "Italian"
Name: "La taqueria Type:
"""

response = cohere_client.chat(
    messages = [
        {
            'role': 'user',
            'content': prompt2
        }
    ],
    model='command-r-plus'
)

response.message.content[0].text

'Name: "La Taqueria" Type: "Mexican"'

Mind the cost increases with the number of examples. You can get info of the costs:

In [39]:
print(response.json(indent=2))

{
  "id": "bbeebabd-4748-4073-a85c-16b79d5abb58",
  "finish_reason": "COMPLETE",
  "message": {
    "role": "assistant",
    "content": [
      {
        "type": "text",
        "text": "Name: \"La Taqueria\" Type: \"Mexican\""
      }
    ]
  },
  "usage": {
    "billed_units": {
      "input_tokens": 114.0,
      "output_tokens": 13.0
    },
    "tokens": {
      "input_tokens": 307.0,
      "output_tokens": 13.0
    }
  }
}


## Use case 2: LLMs as user interface

Natural language can be the new interface. 

The example below asks the user to write a text specifying whether to turn the lights on or off.  


In [45]:
from ipywidgets import widgets

checkbox = widgets.Checkbox(
    value=False,
    description='Lights on',
    disabled=False,
    indent=False
)
checkbox

Checkbox(value=False, description='Lights on', indent=False)

In [47]:
user_input = input("Enter your message")
response = cohere_client.chat(
    messages = [{
        'role': 'system',
        'content': """
The user will write a text about turning off or on the lights.
The user might use different words to describe the action or expressions, 
like turn on, turn off, activate, deactivate, switch on, switch off, etc.
The user might also use different words to describe the lights,
like lights, lamps, bulbs, etc.
Return a single word: "true" if the user wants to turn on the lights, and "false" if the user wants to turn them off.
        """
    },
    {
        'role': 'user',
        'content': user_input
    }
    ],
    model = 'command-r-plus'
)

outcome = response.message.content[0].text
print("(Just for debugging, the output text is", outcome, ")")

if outcome == "true":
    checkbox.value = True
elif outcome == "false":
    checkbox.value = False
else:
    print("Unexpected outcome")

(Just for debugging, the output text is false )


## Use case 3: Structuring data

Use LLMs to extract information and add it to a certain template. 

The next example simulates a text from a health record and how the system would extract the necessary information into a template. 

In [49]:
system_prompt = """
The user will write a text simulating an electronic health record.
Extract the information necessary to populate this json object:
{
  "age": X,
  "temperature": X,
  "reason for visit": "X",
  "tests performed": ["X", "X", "X"],
    "diagnosis": "X"
}
If the information is not present, return "Not found".
Respond only with the json object. 
"""

user_input = input("Enter your message")

response = cohere_client.chat(
    messages = [
      {
        'role': 'system',
        'content': system_prompt  
      }, 
      {
          'role': 'user',
          'content': user_input
      }
    ],
    model='command-r-plus'
)

print(response.message.content[0].text)

{
  "age": 24,
  "temperature": "Not found",
  "reason for visit": "Stomach ache",
  "tests performed": ["Not found"],
  "diagnosis": "Not found"
}


## Use case: Chatbots

This example illustrates the use of a chatbot, where the intent to be fulfilled is accomplished after several messages, and each message needs to include the previous chat history. 

Let's do it in the wrong way first: 

In [16]:
message1 = {'role': 'user', 'content': 'Who discovered polonium and radium?'}
message2 = {'role': 'user', 'content': 'When was she born?'}

response1 = cohere_client.chat(messages=[message1], model='command-r-plus')
response2 = cohere_client.chat(messages=[message2], model='command-r-plus')

print(response2.message.content[0].text)

I'm sorry, but I don't know who you are referring to. Could you please provide more context or details about the person you are asking about?


In [18]:
response1.message

AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text="Marie Curie, a Polish and naturalized-French physicist and chemist, discovered polonium and radium. She discovered these elements in 1898 while studying the cause of pitchblende's radioactivity.")], citations=None)

We need to add the previous chat history, that was stored already in the response json:

In [20]:
response = cohere_client.chat(
    messages=[
        message1,
        response1.message,
        message2
    ],
    model='command-r-plus'
)
print(response.message.content[0].text)


Marie Curie was born on November 7, 1867, in Warsaw, Poland.


In [21]:
response

ChatResponse(id='0aa9da3b-4c1b-4337-aca1-76a5e14560e1', finish_reason='COMPLETE', prompt=None, message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Marie Curie was born on November 7, 1867, in Warsaw, Poland.')], citations=None), usage=Usage(billed_units=UsageBilledUnits(input_tokens=55.0, output_tokens=20.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=254.0, output_tokens=20.0)), logprobs=None)